# Training 

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras_core
import matplotlib.pyplot as plt

Using PyTorch backend.


In [4]:
train_ds = keras_core.utils.image_dataset_from_directory(
    directory='dataset/training_data',
    color_mode='rgb',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256))


validation_ds, testing_ds = keras_core.utils.image_dataset_from_directory(
    directory='dataset/validation_data',
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.5,
    subset='both',
    seed=123,
    batch_size=16,
    image_size=(256, 256))


print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of validation samples: {validation_ds.cardinality()}")
print(f"Number of testing samples: {testing_ds.cardinality()}")

2023-10-07 06:55:07.147498: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 06:55:07.187027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-07 06:55:07.187066: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-07 06:55:07.187091: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-07 06:55:07.196005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.
Using 656 files for training.
Using 655 files for validation.
Number of training samples: 357
Number of validation samples: 41
Number of testing samples: 41


2023-10-07 06:55:11.605202: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
base_model = keras_core.applications.efficientnet.EfficientNetB3(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False,
    pooling="max",
)


model = keras_core.Sequential(
    [   
        keras_core.layers.Input(shape=(256, 256, 3)),
        base_model,
        keras_core.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
        keras_core.layers.Dense(
            256,
            kernel_regularizer=keras_core.regularizers.l2(0.016),
            activity_regularizer=keras_core.regularizers.l1(0.006),
            bias_regularizer=keras_core.regularizers.l1(0.006),
            activation="relu",
        ),
        keras_core.layers.Dropout(rate=0.45, seed=123),
        keras_core.layers.Dense(4, activation="softmax"),
    ]
)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)              │ 10,783,535 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_2           │ (None, 1536)              │      6,144 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_4 (Dense)                 │ (None, 256)               │    393,472 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_2 (Dropout)             │ (None, 256)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ (None, 4)                 │      1,028 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 11,184,179 (42.66 MB)

 Trainable params: 11,093,804 (42.32 MB)

 Non-trainable params: 90,375 (353.03 KB)

In [8]:
model.compile(
    keras_core.optimizers.Adamax(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds, batch_size=16)

Epoch 1/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 115s 321ms/step - accuracy: 0.5410 - loss: 10.0441 - val_accuracy: 0.2485 - val_loss: 6.8295
Epoch 2/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 117s 327ms/step - accuracy: 0.4364 - loss: 5.8340 - val_accuracy: 0.3872 - val_loss: 4.8454
Epoch 3/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 147s 412ms/step - accuracy: 0.4702 - loss: 4.3676 - val_accuracy: 0.5061 - val_loss: 4.8187
Epoch 4/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 115s 322ms/step - accuracy: 0.5031 - loss: 3.2944 - val_accuracy: 0.5290 - val_loss: 2.7069
Epoch 5/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 116s 323ms/step - accuracy: 0.6378 - loss: 2.4174 - val_accuracy: 0.5564 - val_loss: 1.9742
Epoch 6/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 115s 323ms/step - accuracy: 0.6816 - loss: 1.8354 - val_accuracy: 0.8476 - val_loss: 1.4386
Epoch 7/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 115s 322ms/step - accuracy: 0.7302 - loss: 1.4822 - val_accuracy: 0.7713 - val_loss: 1.1745
Epoch 8/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 116s 325ms/step - accuracy: 0.7601 

In [9]:
model.evaluate(testing_ds)

41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.9925 - loss: 0.3512


[0.3595602810382843, 0.989313006401062]

In [10]:
model.save_weights('weight2.weights.h5')